In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#%cd /content/drive/My Drive/MLDL2024_project1-Enrico

In [3]:
import os
import torch
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets.cityscapes import CityscapesCustom
from models.deeplabv2.deeplabv2 import get_deeplab_v2
import torch.optim as optim

In [4]:
#Set device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
#Create Dataloaders for Cityscapes
cityscapes_dir = os.path.dirname(os.getcwd()) + '/Cityscapes/Cityspaces/'

train_dataset = CityscapesCustom(cityscapes_dir, 'train')
test_dataset = CityscapesCustom(cityscapes_dir, 'val')

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

print(f'Train: {len(train_dataset)} images, divided into {len(train_dataloader)} batches of size {train_dataloader.batch_size}')
print(f'Test: {len(test_dataset)} images, divided into {len(test_dataloader)} batches of size {test_dataloader.batch_size}')

Train: 1572 images, divided into 393 batches of size 4
Test: 500 images, divided into 125 batches of size 4


In [6]:
#Set up the model with the pretrained weights
model = get_deeplab_v2().to(device)

Deeplab pretraining loading...


In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [8]:
#Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [9]:
def train_model(model, criterion, optimizer, train_dataloader, test_dataloader, n_epochs=1):
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0.0
        for i, (inputs, labels) in enumerate(train_dataloader):
            if i == 1:
                break
            inputs, labels = inputs.to(device), labels.to(device)
            print(f'Inputs train: {inputs.shape}, {inputs.dtype}')
            print(f'Labels train: {labels.shape}, {labels.dtype}')
            optimizer.zero_grad()
            outputs, _, _ = model(inputs)
            print(f'Outputs train: {outputs.shape}, {outputs.dtype}')

            loss = criterion(outputs, labels.squeeze(1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
        train_loss /= len(train_dataloader)
        
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for i, (inputs, labels) in enumerate(test_dataloader):
                if i == 1:
                    break
                inputs, labels = inputs.to(device), labels.to(device)
                #print(f'Inputs test: {inputs.shape}, {inputs.dtype}')
                #print(f'Labels test: {labels.shape}, {labels.dtype}')
                outputs = model(inputs)
                #print(f'Outputs test: {outputs.shape}, {outputs.dtype}')

                loss = criterion(outputs, labels.squeeze(1))
                test_loss += loss.item()
                
            test_loss /= len(test_dataloader)
        
        print(f'Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

In [10]:
#Train the model
train_model(model, criterion, optimizer, train_dataloader, test_dataloader)

Inputs train: torch.Size([4, 3, 512, 1024]), torch.float32
Labels train: torch.Size([4, 1, 512, 1024]), torch.int64
Outputs train: torch.Size([4, 19, 512, 1024]), torch.float32
Inputs test: torch.Size([4, 3, 512, 1024]), torch.float32
Labels test: torch.Size([4, 1, 512, 1024]), torch.int64
Outputs test: torch.Size([4, 19, 512, 1024]), torch.float32
Epoch 1/1, Train Loss: 0.0180, Test Loss: 0.0026


In [ ]:
#IoU metrics calculation
prediction = torch.argmax(outputs, dim=1)
intersection = np.logical_and(target, prediction)
union = np.logical_or(target, prediction)
iou_score = np.sum(intersection) / np.sum(union)